Extracting the sentiment versus estimated gender of participants on the mailing list. 

In [1]:
%matplotlib inline

Import the BigBang modules as needed. These should be in your Python environment if you've installed BigBang correctly.

In [2]:
import bigbang.ingress.mailman as mailman
import bigbang.analysis.graph as graph
import bigbang.analysis.process as process
from bigbang.parse import get_date
from bigbang.archive import Archive
from bigbang.archive import load as load_archive
import bigbang.analysis.utils as analysis_utils

import imp
imp.reload(process)

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import math
import pytz
import pickle
import os

from bigbang import parse
from gender_detector import gender_detector as gd

from textblob import TextBlob

/tmp/ipykernel_33443/3698918106.py:9: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


Now let's load the data for analysis.

In [118]:
stem = False


import os 
cwd = os.getcwd()    

archives_names = ["httpbisa"]


archive_paths = list()
for archive_name in archives_names:
    archive_paths.append('../archives/'+archive_name+'.csv')
    

archives_list = [load_archive(arch_path).data for arch_path in archive_paths]
    
archives = Archive(pd.concat(archives_list))
archives_data = archives.data


For each of our lists, we'll clean up the names, find the first name if there is one, and guess its gender. Pandas groups the data together for comparison. We keep count of the names we find that are ambiguous, for the next step.

In [119]:
detector = gd.GenderDetector('us')


def guess_gender(name):
    if not name:
        return 'name unknown'
    try:
        if detector.guess(name) == 'unknown':
            return 'gender unknown'
        
        return detector.guess(name)
    except:
        return 'error'


def lookup_index(df, mdf, val, field = 'From Gender'):
    #addr = analysis_utils.extract_email(val)
    #if not addr:
    #    return 'email garbled'
    #else:
        #vidx =  df[[analysis_utils.extract_email(e) in addr for e in df.index]]
    vidx = df[df.index == val]
    if vidx.empty :
        return None
    else :
        #print(vidx.index.values[0])
        midx = mdf[mdf.index == vidx.index.values[0]]
        #print(midx.head())
        #print(midx.index)
        if midx.empty :
            return None
        else :
            return midx[field].values[0]

In [120]:
mod_data = archives_data.dropna()
mod_data = mod_data[mod_data['In-Reply-To'] != 'None']
#mod_data = mod_data.head(30)
mod_data['Sentiment'] = mod_data['Body'].map(lambda x: TextBlob(x).sentiment.polarity)
mod_data['From Name'] = mod_data['From'].map(lambda x: parse.clean_from(x))
mod_data['From First Name'] = mod_data['From Name'].map(lambda x: parse.guess_first_name(x))
mod_data['From Gender'] = mod_data['From First Name'].map(guess_gender)
mod_data['InReplyTo Gender'] = mod_data['In-Reply-To'].map(lambda x: lookup_index(archives_data, mod_data,x))
mod_data = mod_data.dropna()

In [121]:
mod_data['From'].size

25796

In [122]:
archives_data['From'].size

40147

In [123]:
#mod_data
#mod_data['In-Reply-To'].head().map(lambda x: lookup_index(archives_data,x))

tempdata = mod_data.groupby(['From Gender','InReplyTo Gender','From'], as_index=False)['From'].count()

tempdata = pd.DataFrame(tempdata)
#print(tempdata)

#tempdata = mod_data.drop_duplicates('From')

print(tempdata[tempdata['From Gender'] == 'female']['From'].sum())

print(tempdata[tempdata['From Gender'] == 'female'].size)

print(tempdata[tempdata['From Gender'] == 'male']['From'].sum())

print(tempdata[tempdata['From Gender'] == 'male'].size)

print(tempdata[tempdata['InReplyTo Gender'] == 'female']['From'].sum())

print(tempdata[tempdata['InReplyTo Gender'] == 'female'].size)

print(tempdata[tempdata['InReplyTo Gender'] == 'male']['From'].sum())

print(tempdata[tempdata['InReplyTo Gender'] == 'male'].size)

print(tempdata[(tempdata['From Gender'] == 'female') & (tempdata['InReplyTo Gender'] == 'male')]['From'].sum())

print(tempdata[(tempdata['From Gender'] == 'female') & (tempdata['InReplyTo Gender'] == 'male')].size)

print(tempdata[(tempdata['From Gender'] == 'male') & (tempdata['InReplyTo Gender'] == 'female')]['From'].sum())

print(tempdata[(tempdata['From Gender'] == 'male') & (tempdata['InReplyTo Gender'] == 'female')].size)

print(tempdata[(tempdata['From Gender'] == 'male') & (tempdata['InReplyTo Gender'] == 'male')]['From'].sum())

print(tempdata[(tempdata['From Gender'] == 'male') & (tempdata['InReplyTo Gender'] == 'male')].size)

print(tempdata[(tempdata['From Gender'] == 'female') & (tempdata['InReplyTo Gender'] == 'female')]['From'].sum())

print(tempdata[(tempdata['From Gender'] == 'female') & (tempdata['InReplyTo Gender'] == 'female')].size)

328
150
16474
3423
323
270
15604
2202
190
75
183
195
10002
1566
6
12


In [124]:
print("Minimun Sentiment Expressed from one gender to another")
print(mod_data.groupby(['From Gender', 'InReplyTo Gender'])['Sentiment'].aggregate('min'))
print("=======")
print("=======")
print("Maximum Sentiment Polarity Expressed from one Gender to another")
print(mod_data.groupby(['From Gender', 'InReplyTo Gender'])['Sentiment'].aggregate('max'))
print("=======")
print("=======")
print("Sum of all sentiment polarity expressed from one Gender to another")
print(mod_data.groupby(['From Gender', 'InReplyTo Gender'])['Sentiment'].aggregate('sum'))
print("=======")
print("=======")
print("Average sentiment polarity expressed from one Gender to another")
print(mod_data.groupby(['From Gender', 'InReplyTo Gender'])['Sentiment'].aggregate('mean'))
print("=======")
print("=======")
print("Standard Deviation in sentiment polarity expressed from one Gender to another")
print(mod_data.groupby(['From Gender', 'InReplyTo Gender'])['Sentiment'].aggregate('std'))
print("=======")
print("=======")
print("Variance in sentiment polarity expressed from one Gender to another")
print(mod_data.groupby(['From Gender', 'InReplyTo Gender'])['Sentiment'].aggregate('var'))
print("=======")


Minimun Sentiment Expressed from one gender to another
From Gender     InReplyTo Gender
error           error               0.154949
                gender unknown      0.045000
                male                0.065530
female          female             -0.114744
                gender unknown     -0.087662
                male               -0.121667
                name unknown       -0.090000
gender unknown  error               0.077412
                female             -0.050000
                gender unknown     -0.750000
                male               -1.000000
                name unknown       -0.205556
male            error               0.047436
                female             -0.462500
                gender unknown     -0.700000
                male               -0.750000
                name unknown       -0.175000
Name: Sentiment, dtype: float64
Maximum Sentiment Polarity Expressed from one Gender to another
From Gender     InReplyTo Gender
error           er